In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(10,7))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack import qubicpack as qp
import fibtools as ft
import plotters as p

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string

In [ ]:
rc('figure',figsize=(10,7))

Get the directories corresponding to the day we consider:

In [ ]:
day = '2019-02-22'
data_dir = '/qubic/Data/Calib-TD/'+day+'/'
dirs = glob.glob(data_dir+'/*')
dirs

In [ ]:
labels = []
for d in dirs:
    bla = str.split(d,'__')
    print bla
    labels.append(bla[1])
labels

So we first look at pixel 63 in ASIC2 in first dataset (Source_scan1). Same pixel as Michel did on QubicStudio,  read the Azimuth and put both times starting at zero

In [ ]:
thedir = dirs[0]
a = qp()
a.read_qubicstudio_dataset(thedir, asic=2)
nsamples = len(a.timeline(TES=63))
pow2 = int(np.log(nsamples)/np.log(2))
ns = 2**pow2
FREQ_SAMPLING = 1./a.sample_period()

t_data = a.timeline_timeaxis(axistype='pps')
#t_data = a.timeline_timeaxis(axistype='index')

t_az, azinit = ft.read_hkintern(thedir,thefieldname='Platform-Azimut')
az = (azinit-2.**15)/2.**16*360

mini_t_data = np.min(t_data)
t_data = t_data-mini_t_data
t_az = t_az-mini_t_data

In [ ]:
# print(told)
# print(tnew-tnew[0])
# TESnum = 63
# data = a.timeline(TES=TESnum)
# plot(told, data, alpha=0.5)
# plot(tnew-np.min(tnew)-122, data, alpha=0.5)

## Calibration of the number of step motor per degree 

In [ ]:
angle_given = 10.
angle_measured = np.max(az) - np.min(az)
print(angle_given, angle_measured)

nb_encoder_step_per_degree = (np.max(azinit) - np.min(azinit)) / angle_measured
print(nb_encoder_step_per_degree)

nb_motor_step_per_degree = 168830
nb_motor_step_per_degree_new = nb_motor_step_per_degree / (angle_measured / angle_given)
print (nb_motor_step_per_degree_new)

In [ ]:
print(t_data)
print(t_az)

TESnum = 63
data = a.timeline(TES=TESnum)
plot(t_data,(data-np.mean(data))/np.std(data),label='TES')
plot(t_az-np.min(t_az), (az-np.mean(az))/np.std(az),label='Az')
legend()
print(np.min(az),np.max(az))

thedata = data - f.gaussian_filter1d(data, 1000)

In [ ]:
new_az = np.interp(t_data, t_az-min(t_az), az)
plot(new_az, data,'b,')
xlabel('Azimuth [deg]')
ylabel('Data')

In [ ]:
ppp = 3.
period = ppp*(t_data/int(ppp)).astype(int)
allperiods = np.unique(period)
tper = np.zeros(len(allperiods))
azper = np.zeros(len(allperiods))
ampdata = np.zeros(len(allperiods))
for i in xrange(len(allperiods)):
    ok = (period == allperiods[i])
    azper[i] = np.mean(new_az[ok])
    tper[i] = np.mean(t_data[ok])
    ampdata[i] = np.std(data[ok])
plot(tper,ampdata,'b')

In [ ]:
plot(azper,ampdata,'b')

In [ ]:
elevation = 50.
nbins = 180
azfinalbins = min(azper)+(max(azper)-min(azper))*np.linspace(0,1,nbins+1)
angfinal = np.zeros(nbins)
ampfinal = np.zeros(nbins)
err_ampfinal = np.zeros(nbins)
for i in xrange(nbins):
    ok = (azper >= azfinalbins[i]) & (azper < azfinalbins[i+1])
    angfinal[i] = 0.5 * (azfinalbins[i] + azfinalbins[i+1]) * np.cos(np.radians(elevation))
    ampfinal[i] = np.mean(ampdata[ok])
    err_ampfinal[i] = np.std(ampdata[ok])/np.sqrt(ok.sum())

def gauss(x,par):
    return par[3]+par[2]*np.exp(-0.5 * (x-par[0])**2 / par[1]**2)

thecut = 350000
okfit = (ampfinal < thecut) & (err_ampfinal != 0)
print(okfit.sum())
guess = np.array([3., 2., np.max(ampfinal)-np.min(ampfinal), np.min(ampfinal)])
res = ft.do_minuit(angfinal[okfit], ampfinal[okfit], err_ampfinal[okfit], guess, functname=gauss, verbose=False)

rc('figure',figsize=(12,7))

subplot(1,2,1)
plot(angfinal, ampfinal,'b',label='Data')
plot(angfinal[okfit], ampfinal[okfit],'b.',label='Data for fit')
plot(angfinal, gauss(angfinal, res[1]), 'r', label='Fit: FWHM={0:4.2f} deg'.format(res[1][1]*2.35))
plot(angfinal, angfinal*0+thecut, 'k:',label='Cut for fit')
xlabel('Angle on Sky')
ylabel('Beam')
title('TES #{}'.format(TESnum))
legend(fontsize=9)

ampfinaldB = 10 * np.log10(np.abs((ampfinal-res[1][3])/res[1][2]))
subplot(1,2,2)
plot(angfinal, ampfinaldB,'b')
plot(angfinal[okfit], ampfinaldB[okfit],'b.')
plot(angfinal, 10*np.log10((gauss(angfinal, res[1])-res[1][3])/res[1][2]), 'r', label='Fit')
xlabel('Angle on Sky')
ylabel('Beam [dB]')
title('TES #{}'.format(TESnum))
ylim(-25,0)
tight_layout()

In [ ]:
fcalsrc = np.sort(glob.glob('/qubic/Data/Calib-TD/calsource/calsource_20190222*'))
print fcalsrc
tt = []
dd = []
for ff in fcalsrc:
    thett, thedd = np.loadtxt(ff).T
    tt.append(thett)
    dd.append(thedd)

tt = np.concatenate(tt)
dd = np.concatenate(dd)
tt += 3600-mini_t_data
dd = (dd-np.mean(dd))/np.std(dd)
plot(tt,dd,',')
ylim(-1,1)

In [ ]:

fct = np.sin((t_data+0.75)*2*np.pi/(1./0.333))/2

shift = -0.45
subplot(2,1,1)
plot(t_data, thedata/np.max(thedata), alpha=0.5, label='data')
plot(tt-np.min(t_az)-shift, dd, alpha=0.5, label='Source')
plot(t_data, fct, alpha=0.5, label='Sinewave on t_data')
ylim(-1,1)
legend(loc='lower left')

subplot(2,3,4)
plot(t_data, thedata/np.max(thedata))
plot(tt-np.min(t_az)-shift, dd)
plot(t_data, fct)
ylim(-1,1)
xlim(520,530)

subplot(2,3,5)
plot(t_data, thedata/np.max(thedata))
plot(tt-np.min(t_az)-shift, dd)
plot(t_data, fct)
ylim(-1,1)
xlim(1900,1910)

subplot(2,3,6)
plot(t_data, thedata/np.max(thedata))
plot(tt-np.min(t_az)-shift, dd)
plot(t_data, fct)
ylim(-1,1)
xlim(4790,4800)
